In [318]:
import math
import string
import gensim.downloader as api
import torch
from nltk import word_tokenize, MWETokenizer
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertForMaskedLM
from nltk.corpus import wordnet as wn
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

In [319]:
w2v_model = api.load('word2vec-google-news-300')

In [320]:
nlp = English()
tokenizer = Tokenizer(nlp.vocab)

In [133]:
stop_words = set(stopwords.words('english'))
stop_words.update('.', '?', '-', '\'', '\:', ',', '!', '<', '>', '\"', '/', '(', ')',
                  '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 's', 't', 're', 'm')

In [399]:
print(w2v_model.similarity('mouse', 'hamster'))
print(w2v_model.similarity('mouse', 'owl'))

0.4880635
0.29449567


In [479]:
lst =[1,2,3,4,5,6,7,8,9]
lst2 = (lst[0:3]+lst[-3:])
print(lst2)

[1, 2, 3, 7, 8, 9]


In [510]:
def get_possible_pun_words(sent):
    scores = {}
    print('len',len(sent))
    if len(sent) <= 1:
        return set(sent[0])
    else:
        for i in range(len(sent)-1):
            for j in range(i+1, len(sent)):
                sim_score = w2v_model.similarity(sent[i], sent[j])
                scores['{0}-{1}'.format(sent[i], sent[j])] = sim_score
        print('scores',scores)
        if len(scores) >= 5:
            top3 = sorted(zip(scores.values(), scores.keys()), reverse=True)[0:3]
            bottom3 = sorted(zip(scores.values(), scores.keys()), reverse=False)[:3]
            final = top3+bottom3
#             final = list(final)
#             final = tuple(final)
#             print('')
#             print('bottom3',bottom3)
#             print('')
            print('final',final)
            print('')
            print('top3',top3)
            print(" ")
            poss = [tup[1].split(sep='-') for tup in top3]
#             poss = [tup[1].split(sep='-') for tup in final]

            print('poss',poss)
            print(" ")
            possible_pun_words = set(poss[0] + poss[1] + poss[2])
#             possible_pun_words = set(poss[0] + poss[1] + poss[2]+poss[3]+poss[4]+poss[5])
            
            print('possible_pun_words',possible_pun_words)
            print(" ")
        else:
            poss = [pair.split(sep='-') for pair in scores.keys()]
            possible_pun_words = set()
            for i in range(len(poss)):
                possible_pun_words = possible_pun_words.union(set(poss[i]))
            """
            top = sorted(zip(scores.values(), scores.keys()), reverse=True)[:1]
            poss = [tup[1].split(sep='-') for tup in top]
            possible_pun_words = set(poss[0])
            """
            
        return possible_pun_words

In [554]:
def get_pun_word(orig_sent, sent):
    pun_words = get_possible_pun_words(sent)
    pun_words = pun_words
    largest_index = 0
    print('orig_sent',orig_sent)
    for w in pun_words:
#         pun_word = w
        index = orig_sent.index(w)
        if index > largest_index:
            largest_index = index
#     return largest_index + 1
    punword=orig_sent[largest_index+1]
    print('lagrest_index',largest_index)
#     print(orig_sent[13])
    return punword

In [555]:
def mask_and_predict(sentence):
 # Load pre-trained model and tokenizer
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    encoding = tokenizer.encode(sentence)
    words = tokenizer.convert_ids_to_tokens(encoding)
    words = words [1:-1]
    words = sentence.split()
#     words = nlp(sentence)
    original_sentences = []
    original_sentence = []

    results = []
    print(words)
    vocab = w2v_model.index_to_key

    
    for i in range(len(words)):
        original_sentence.append(words[i].lower())
    original_sentences.append(original_sentence)

    sentences = []
    sentence = []

    for i in range(len(words)):
        if words[i].lower() not in stop_words and words[i].lower() in vocab:
            sentence.append(words[i].lower())
#             print(i, sentence)
    sentences.append(sentence)
    
    print('original_sentences',original_sentences[0])
    print(" ")
    print('sentences',sentences[0])
    
    get_pun_words_list = []
    text_list = []
    
    for i in range(len(sentences)):
        get_pun_words_result = get_pun_word(original_sentences[i], sentences[i])
        get_pun_words_list.append(get_pun_words_result)
    print(get_pun_words_list)
#     print(text_list)

In [556]:
s = "I've got a bad case of shingles"
s2  = 'Autobiography when your car starts telling you about its life. '
s3 = "She was only a Blacksmith's daughter, but she knew how to forge ahead ."
s4 = "A ditch digger was entrenched in his career ."
s5 ="OLD GEOGRAPHERS never die , they just become legends ."
s6 ="The young monkeys went to the jungle gym for some exercise"
s7 = "The investor in the bakery demanded a larger piece of the pie."
s8 = "The art competition ended in a draw."
s9 = "The maestro turned away from the orchestra as they told him the bad news; he couldn’t face the music."

In [557]:
mask_and_predict(s5)

['OLD', 'GEOGRAPHERS', 'never', 'die', ',', 'they', 'just', 'become', 'legends', '.']
original_sentences ['old', 'geographers', 'never', 'die', ',', 'they', 'just', 'become', 'legends', '.']
 
sentences ['old', 'geographers', 'never', 'die', 'become', 'legends']
len 6
scores {'old-geographers': -0.026032887, 'old-never': 0.117951, 'old-die': 0.2354283, 'old-become': 0.097501636, 'old-legends': 0.086471416, 'geographers-never': 0.04967519, 'geographers-die': 0.046451222, 'geographers-become': 0.05736023, 'geographers-legends': 0.13373458, 'never-die': 0.20109296, 'never-become': 0.22373563, 'never-legends': 0.15837559, 'die-become': 0.21707109, 'die-legends': 0.08356403, 'become-legends': -0.01524009}
final [(0.2354283, 'old-die'), (0.22373563, 'never-become'), (0.21707109, 'die-become'), (-0.026032887, 'old-geographers'), (-0.01524009, 'become-legends'), (0.046451222, 'geographers-die')]

top3 [(0.2354283, 'old-die'), (0.22373563, 'never-become'), (0.21707109, 'die-become')]
 
poss [['

In [543]:
def get_word_with_most_senses(sentence):
#     model_name = 'bert-base-uncased'
#     tokenizer = BertTokenizer.from_pretrained(model_name)
#     encoding = tokenizer.encode(sentence)
#     words = tokenizer.convert_ids_to_tokens(encoding)
#     words = words [1:-1]
    words = sentence.split()
    print(words)
#     text_id = words.attrib['id']
#               words.index()
    for i in range(len(words)):
        senses_number = 0
        prev_word_id = ""
        if words[i].lower() not in stop_words:
            word_senses_num = len(wn.synsets(words[i].lower()))
            print(" ")
            print('wordsenses_num',word_senses_num)
#             print('synsets', wn.synsets(child[i].text.lower()))
            
            print('synsets', wn.synsets(words[i].lower()))
#             current_word_id = child[i].attrib['id']
            current_word_id = words[i]
            if word_senses_num >= senses_number:
                senses_number = word_senses_num
                prev_word_id = current_word_id
    pun_prediction = prev_word_id
    print(pun_prediction)

In [548]:
get_word_with_most_senses(s5)

['OLD', 'GEOGRAPHERS', 'never', 'die', ',', 'they', 'just', 'become', 'legends', '.']
 
wordsenses_num 9
synsets [Synset('old.n.01'), Synset('old.a.01'), Synset('old.a.02'), Synset('old.s.03'), Synset('old.s.04'), Synset('erstwhile.s.01'), Synset('honest-to-god.s.01'), Synset('old.s.07'), Synset('previous.s.01')]
 
wordsenses_num 1
synsets [Synset('geographer.n.01')]
 
wordsenses_num 2
synsets [Synset('never.r.01'), Synset('never.r.02')]
 
wordsenses_num 14
synsets [Synset('die.n.01'), Synset('die.n.02'), Synset('die.n.03'), Synset('die.v.01'), Synset('die.v.02'), Synset('die.v.03'), Synset('fail.v.04'), Synset('die.v.05'), Synset('die.v.06'), Synset('die.v.07'), Synset('die.v.08'), Synset('die.v.09'), Synset('die.v.10'), Synset('die.v.11')]
 
wordsenses_num 4
synsets [Synset('become.v.01'), Synset('become.v.02'), Synset('become.v.03'), Synset('become.v.04')]
 
wordsenses_num 2
synsets [Synset('legend.n.01'), Synset('caption.n.03')]



In [148]:

def get_definitions(word):
    synsets = wn.synsets(word)
    definitions = [syn.definition() for syn in synsets]
    return definitions

def context_relevant_definition(word, context_word):
    context_synsets = wn.synsets(context_word)
    word_synsets = wn.synsets(word)

    # If either word has no synsets, return None
    if not context_synsets or not word_synsets:
        return None

    best_definition = None
    max_similarity = -1

    for w_syn in word_synsets:
        for c_syn in context_synsets:
            similarity = wn.wup_similarity(w_syn, c_syn)
            if similarity is not None and similarity > max_similarity:
                max_similarity = similarity
                best_definition = w_syn.definition()

    return best_definition, max_similarity  # Return tuple with definition and similarity


def compute_similarity(word1, word2):
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)

    # if there are no synsets, return 0
    if not synsets1 or not synsets2:
        return 0

    best_score = 0
    # iterate over pairs of synsets and compute their path similarity
    for synset1 in synsets1:
        for synset2 in synsets2:
            similarity = synset1.path_similarity(synset2)
            if similarity is not None and similarity > best_score:
                best_score = similarity

    return best_score


def sigmoid(x):
    """Returns the sigmoid of x."""
    return 1 / (1 + math.exp(-x))


def definition_avg_similarity(definition, context_word):
    """
    Compute the average similarity of words in the definition to the context word.

    :param definition: The definition text.
    :param context_word: The context word to compare against.
    :param model: Preloaded Word2Vec model.
    """
    tokens = [word for word in definition.lower().split() if word.isalnum()]

    total_similarity = 0
    valid_token_count = 0
    for token in tokens:
        if token in model:
            similarity = model.similarity(context_word, token)
            total_similarity += similarity
            valid_token_count += 1

    avg_similarity = total_similarity / valid_token_count if valid_token_count else 0

    return avg_similarity


def context_relevant_definition(word, context_word):
    word_synsets = wn.synsets(word)
    if not word_synsets:
        return None

    best_definition = None
    max_avg_similarity = -1
    for w_syn in word_synsets:
        current_avg_similarity = definition_avg_similarity(w_syn.definition(), context_word)
        print(f"{context_word} Definition: '{w_syn.definition()}' has average similarity: {current_avg_similarity}")
        if current_avg_similarity > max_avg_similarity:
            max_avg_similarity = current_avg_similarity
            best_definition = w_syn.definition()

    return best_definition

In [7]:
def main(sentence):
    tokenizer = MWETokenizer()  # Multi-word expression tokenizer
    tokens = tokenizer.tokenize(word_tokenize(sentence))
    # definition = context_relevant_definition(word, context_words)

    predictions = [x for x in mask_and_predict(" ".join(tokens))]
    print(predictions)
    predictions.sort(key=lambda x: x[2], reverse=False)
    filtered_data = predictions[:3]
    print(filtered_data)
    if filtered_data[0][2] < 0.2:
        print(f"Sentence to detect pun: {sentence}")
        # find element the lowest probability
        pun_word = min(filtered_data, key=lambda x: x[2])
        if len(wn.synsets(pun_word[0])) <2:
            pun_word = filtered_data[1]
        filtered_data.remove(pun_word)
        # pun_word = pun_word[0]
        print(f"This is a pun! The word '{pun_word[0]}' has multiple meanings.")
        if len(wn.synsets(pun_word[0])) == 2:
            for index, definition in enumerate(wn.synsets(pun_word[0])):
                print(f"Definition {index + 1}: {definition.definition()}. ", end="")
                print(' ')

            return
        # for index, item in enumerate(filtered_data):
        #     print(f"Definition {index + 1}: {context_relevant_definition(pun_word, item[0])}. ", end="")
        from test2 import max_similarity
        for index, item in enumerate(filtered_data):
            # print(max_similarity(pun_word[0], item[0]))
            print(f"Definition {index + 1}: {max_similarity(pun_word[0], item[0])[0][0]}. ", end="")
            print(' ')

    else:
        print("This is not a pun!")


if __name__ == '__main__':
    main("Can honeybee abuse lead to a sting operation ?")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[('Can', 'could', 0.37111154198646545), ('honeybee', 'sexual', 0.22620375454425812), ('abuse', '##s', 0.5341664552688599), ('lead', 'lead', 0.8986830711364746), ('sting', 'successful', 0.1102379709482193), ('operation', 'attack', 0.16788507997989655)]
[('sting', 'successful', 0.1102379709482193), ('operation', 'attack', 0.16788507997989655), ('honeybee', 'sexual', 0.22620375454425812)]
Sentence to detect pun: Can honeybee abuse lead to a sting operation ?
This is a pun! The word 'sting' has multiple meanings.


ModuleNotFoundError: No module named 'test2'